MIT License

Copyright (c) 2017 Ryan Whytsell

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


Modified by Harish Iyer Ganesan

In [1]:
import sys
import pygame
import random
from threading import Thread
#import PreyPredator
import time

import warnings
warnings.filterwarnings("ignore")

pygame 2.1.0 (SDL 2.0.16, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import random
import sys
import copy
random.seed(1)

In [3]:
import csv

In [4]:
class Node:

    species = None
    x = None
    y = None
    health = None
    

    def __init__(self, spec=None):
        if spec is not None:
            self.species = spec
        else:
            self.species = 0
        self.x = None
        self.y = None
        #self.health = self.species*2
        if spec is 0:  # Blank
            self.health = None
        if spec is 1:  # Predator
            self.health = 2
        if spec is 2:  #Intermediate-Predator
            self.health = 3
        if spec is 3:  # Prey
                self.health = 4

    def set_location(self, pos_x, pos_y):
        self.x = pos_x
        self.y = pos_y

    def set_species(self, num):
        if num == 0 or num == 1 or num == 2 or num == 3:
            self.species = num
            if num is 0:  # Blank
                self.health = None
            if num is 1:  # Predator
                self.health = 2
            if num is 2:  #Intermediate-Predator
                self.health = 2
            if num is 3:  # Prey
                self.health = 4
        else:
            sys.stderr.write('Species not correct, do not do math with them.')
            self.species = 0

    def move_here(self, species, health):
        if species == 0 or species == 1 or species == 2 or species == 3:
            self.species = species
            if species is 0:  # Blank
                self.health = health
            if species is 1:  # Predator
                self.health = health
            if species is 2:  #Intermediate-Predator
                self.health = health
            if species is 3:  # Prey
                self.health = health
        else:
            sys.stderr.write('Species not correct, do not do math with them.')
            self.species = 0

    def prey_eat(self):
        self.species = 1
        #self.health = 2
            
    def predator_eat_intermediate_predator(self):
        self.species = 1
        #self.health = 2
            
    def intermediate_predator_prey_eat(self):
        self.species = 2
        #self.health = 3

            
    def intermediate_predator_reproduce(self):
        self.species = 2
        #self.health = 3

    def prey_reproduce(self):
        self.species = 3
        self.health = 4

In [5]:
class Map:
    height = 100
    width = 100
    play_board = []
    kill_count = 0
    prey_kill_count = 0
    
    def __init__(self):
        
        prey = 0
        predator = 0
        intermediate_predator = 0
        empty = 0       
        for x in range(self.width):
            row = []
            
            # Random Spatial Dostribution
            for y in range(self.height):
                i = random.randint(0, 100)
                if i <= 25:
                    row.append(Node(3))
                    prey += 1
                elif i <= 50 :
                    row.append(Node(2))
                    intermediate_predator += 1
                elif i<= 75:
                    row.append(Node(1))
                    predator += 1
                else:
                    row.append(Node(0))
                    empty += 1
               
#             #Clustered Spatial Dostribution
#             for y in range(self.height):
#                 i = random.randint(0, 100)
#                 if i <= 25 and x<50 and y<50:
#                     row.append(Node(3))
#                     prey += 1
#                 elif i <= 25 and x>50 and y>50:
#                     row.append(Node(3))
#                     prey += 1
#                 elif i <= 50 and x<50 and y<50:
#                     row.append(Node(2))
#                     intermediate_predator += 1
#                 elif i <= 75 and x>50 and y>50:
#                     row.append(Node(1))
#                     predator += 1
#                 else:
#                     row.append(Node(0))
#                     empty += 1

            self.play_board.append(row)
        print('Map created')
        print('Prey: ', prey)
        print('Intermediate-Predator: ', intermediate_predator)
        print('Predator: ', predator)
        print('Empty: ', empty)
        print('Width: ', len(self.play_board))
        print('Height of row 5: ', len(self.play_board[5]))
        #writer.writerow({'time_step': 0, 'prey': prey, 'intermediate_predator': intermediate_predator, 'predator': predator, 'empty': empty  })
    
    def map_count(self, time_step,writer):
        pb = self.play_board
        prey = 0
        predator = 0
        intermediate_predator = 0
        empty = 0
        for x in range(0, self.width):
            for y in range(0, self.height):
                if  pb[x][y].species is 3:
                    prey += 1
                elif pb[x][y].species is 2:
                    intermediate_predator += 1
                elif pb[x][y].species is 1:
                    predator += 1
                else:
                    empty += 1
                
                
        print('Time Step: ',time_step)
        print('Prey: ', prey)
        print('Intermediate-Predator: ', intermediate_predator)
        print('Predator: ', predator)
        print('Empty: ', empty)
        print('-----------------------------------------')
        writer.writerow({'time_step': time_step, 'prey': prey, 'intermediate_predator': intermediate_predator, 'predator': predator, 'empty': empty  })

    
    def get_board(self):
        return self.play_board

    def turn(self):
        pb = self.play_board
        [[self.check_neighbors(x, y, pb[x][y]) for y in range(0, self.height)] for x in range(0, self.width)]

    # Returns the neighbors of a node, If node is on edge this wraps
    def get_neighbors(self, x, y):
        top = self.play_board[x][(y - 1) % self.height]
        tr = self.play_board[(x + 1) % self.width][(y - 1) % self.height]
        right = self.play_board[(x + 1) % self.width][y]
        br = self.play_board[(x + 1) % self.width][(y + 1) % self.height]
        bottom = self.play_board[x][(y + 1) % self.height]
        bl = self.play_board[(x - 1) % self.width][(y + 1) % self.height]
        left = self.play_board[(x - 1) % self.width][y]
        tl = self.play_board[(x - 1) % self.width][(y - 1) % self.height]

        neighbors = [top, tr, right, br, bottom, bl, left, tl]
        #neighbors = [top, right, bottom, left]
        return neighbors

    #
    # PREDATOR AND PREY CELLULAR AUTOMATON
    # The world is grid of cells, with 4 possibilities: Predator(Red, 1), Intermediate-Predator(Yellow, 2) Prey(Blue, 3), or Empty(Black, 0).
    # All predator, intermediate-predator and prey have a set health, that changes over time.
    # The simulation works in steps, with the following rules:
    #    -For prey:
    #        -Tries to move in a random direction.
    #        -Health increases.
    #        -When health reaches a threshold:
    #           -They will reproduce, creating a new "Prey"
    #            -Their health resets to 1
    #    -For intermediate-predator:
    #        -Tries to move in a random direction.
    #        -Health decreases.
    #        -When health reaches 0, they die and turn into "Nothing".
    #       -If the adjacent square is a prey:
    #            -They will eat it, turning it into a "predator" (reproducing)
    #            -Their health will increase by the amount of health the eaten prey had
    #    -For predator:
    #        -Tries to move in a random direction.
    #        -Health decreases.
    #        -When health reaches 0, they die and turn into "Nothing".
    #       -If the adjacent square is a prey:
    #            -They will eat it, turning it into a "predator" (reproducing)     
    #            -Their health will increase by the amount of health the eaten prey had
    
    def check_neighbors(self, x, y, node):
        # If the node is empty no checks are performed
        if node.species is 0:
            return

        if node.species is 1:
            # This is a predator. Check for prey if no prey reduce health/die
            neighbors = self.get_neighbors(x, y)
            prey = []
            intermediate_predator = []
            open_ = []

            for neigh in neighbors:
                if neigh.species is 3:
                    prey.append(neigh)
                if neigh.species is 2:
                    intermediate_predator.append(neigh)
                if neigh.species is 0:
                    open_.append(neigh)
            open_length = len(open_)

            
            if len(prey) is 0 and len(intermediate_predator) is 0:
                if node.health > 0 and open_length > 0:
                    node.health -= 1
                    open_[random.randint(0, (open_length-1))].move_here(1, node.health)
                    node.set_species(0)
                else:
                    node.health -= 1
                                
            elif len(prey) > 0:
                target = prey[random.randint(0, (len(prey)-1))]
                if node.health >= target.health:
                    target.prey_eat() 
                    self.prey_kill_count+=1
                elif len(intermediate_predator)>0:
                    target = intermediate_predator[random.randint(0, (len(intermediate_predator)-1))]
                    if node.health >= target.health:
                        target.predator_eat_intermediate_predator()
                        self.kill_count+=1
                        
            elif len(intermediate_predator)>0:
                target = intermediate_predator[random.randint(0, (len(intermediate_predator)-1))]
                if node.health >= target.health:
                    target.predator_eat_intermediate_predator()
                else:
                    node.health -= 1
                          
            if node.health is None or node.health <= 0:
                node.set_species(0)
                return

        if node.species is 2:
            # This is an intermediate-predator. Check for prey if no prey reduce health/die
            neighbors = self.get_neighbors(x, y)
            prey = []
            predator = []
            intermediate_predator = []
            open_ = []

            for neigh in neighbors:
                if neigh.species is 3:
                    prey.append(neigh)
                if neigh.species is 2:
                    intermediate_predator.append(neigh)
                if neigh.species is 1:
                    predator.append(neigh)
                if neigh.species is 0:
                    open_.append(neigh)
            open_length = len(open_)
            

            if len(prey) is 0 and len(predator) is 0:
                if node.health >0  and open_length > 0:
                    node.health -= 1
                    open_[random.randint(0, (open_length-1))].move_here(2, node.health)
                    node.set_species(0)
                else:
                    node.health -= 1
            elif len(prey) > 0:
                target = prey[random.randint(0, (len(prey)-1))]
                if node.health >= target.health:
                    target.intermediate_predator_prey_eat()
                    self.prey_kill_count+=1

            elif len(predator)>0 and len(intermediate_predator)>4:
                target = predator[random.randint(0, (len(predator)-1))]
                if node.health >= target.health:
                    target.intermediate_predator_prey_eat()
            else:
                node.health -= 1

            if node.health is None or node.health <= 0:
                node.set_species(0)
                return


        if node.species is 3:
            # Check for overpopulation/health This is prey
            neighbors = self.get_neighbors(x, y)
            open_spots = []
            for neigh in neighbors:
                if neigh.species is 0:
                    open_spots.append(neigh)
            count = len(open_spots)
            if count >= 7 or count < 1:
                node.health -= 2
            if node.health is None or node.health is 0:
                node.set_species(0)
                return
            if node.health > 6 and len(open_spots) >= 1:
                open_spots[random.randint(0, (len(open_spots))-1)].prey_reproduce()
                node.health = 4
                return
            node.health += 1

In [6]:
#random.seed(1)
size = width, height = 500, 500
play_width = width/5
play_height = height/5
is_stopped = False

In [7]:
# These are the codes that pygame uses, I am not sure what format it is
black = 0
red = ((255,0,0))
yellow = ((255,255,0))
blue = ((0,0,255))

color_arr = [black, red, yellow, blue]

In [8]:
pygame.init()
screen = pygame.display.set_mode(size)
pp = Map()

Map created
Prey:  2535
Intermediate-Predator:  2525
Predator:  2500
Empty:  2440
Width:  100
Height of row 5:  100


In [9]:
# Paints a 3x3 square for each node
def paint(x, y, color_int):
    screen.set_at((x, y), color_arr[color_int])
    screen.set_at((x + 1, y), color_arr[color_int])
    screen.set_at((x, y + 1), color_arr[color_int])
    screen.set_at((x + 1, y + 1), color_arr[color_int])
    screen.set_at((x + 2, y), color_arr[color_int])
    screen.set_at((x, y + 2), color_arr[color_int])
    screen.set_at((x + 2, y + 2), color_arr[color_int])
    screen.set_at((x + 2, y + 1), color_arr[color_int])
    screen.set_at((x + 1, y + 2), color_arr[color_int])

In [10]:
# Creates random colors TESTING ONLY
def random_colors(w, h):
    ra = random.randint
    pa = paint
    while not is_stopped:
        for x in range(5, w, 5):
            for y in range(5, h, 5):
                color_int = ra(1, 2)
                pa(x, y, color_int)

In [11]:
# Paints the species at each point on the play board
def paint_map(board):
    pa = paint
    length = len(board)
    for x in range(0, length):
        for y in range(0, length):
            pa(x*5, y*5, board[x][y].species)

In [12]:
#### Game loop, paints the map then calculates the next turn
time_step=0
#open the file in the write mode
with open('1prey_2predator_Symmetric.csv', 'w', newline='') as csvfile:
    fieldnames = ['time_step', 'prey','intermediate_predator','predator','empty']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                is_stopped = True
                sys.exit(0)
        paint_map(pp.get_board())
        pygame.display.flip()
        pp.map_count(time_step, writer)
        pp.turn()
#         if(time_step==0 or time_step==200 or time_step==500 or time_step==1000):
#             a=input("Stop for snapshot")
        if(time_step==0 or time_step==500):
            time.sleep(10)
            print("Kill Count: ", pp.kill_count)
            print("Prey Kill Count: ",pp.prey_kill_count)
            pygame.QUIT
            #a=input("Stop for snapshot")
        time_step+=1
            

Time Step:  0
Prey:  2535
Intermediate-Predator:  2525
Predator:  2500
Empty:  2440
-----------------------------------------
Kill Count:  68
Prey Kill Count:  110
Time Step:  1
Prey:  2425
Intermediate-Predator:  2563
Predator:  2572
Empty:  2440
-----------------------------------------
Time Step:  2
Prey:  2293
Intermediate-Predator:  2583
Predator:  2465
Empty:  2659
-----------------------------------------
Time Step:  3
Prey:  2270
Intermediate-Predator:  2496
Predator:  2308
Empty:  2926
-----------------------------------------
Time Step:  4
Prey:  3904
Intermediate-Predator:  2561
Predator:  2267
Empty:  1268
-----------------------------------------
Time Step:  5
Prey:  3342
Intermediate-Predator:  3041
Predator:  2302
Empty:  1315
-----------------------------------------
Time Step:  6
Prey:  2297
Intermediate-Predator:  3941
Predator:  2380
Empty:  1382
-----------------------------------------
Time Step:  7
Prey:  2084
Intermediate-Predator:  4606
Predator:  2240
Empty:  1

Time Step:  67
Prey:  1949
Intermediate-Predator:  2093
Predator:  402
Empty:  5556
-----------------------------------------
Time Step:  68
Prey:  1893
Intermediate-Predator:  1996
Predator:  396
Empty:  5715
-----------------------------------------
Time Step:  69
Prey:  1811
Intermediate-Predator:  1933
Predator:  417
Empty:  5839
-----------------------------------------
Time Step:  70
Prey:  1717
Intermediate-Predator:  1911
Predator:  431
Empty:  5941
-----------------------------------------
Time Step:  71
Prey:  1629
Intermediate-Predator:  1886
Predator:  408
Empty:  6077
-----------------------------------------
Time Step:  72
Prey:  1541
Intermediate-Predator:  1882
Predator:  364
Empty:  6213
-----------------------------------------
Time Step:  73
Prey:  1507
Intermediate-Predator:  1778
Predator:  349
Empty:  6366
-----------------------------------------
Time Step:  74
Prey:  1499
Intermediate-Predator:  1676
Predator:  339
Empty:  6486
----------------------------------

Time Step:  132
Prey:  2199
Intermediate-Predator:  1481
Predator:  273
Empty:  6047
-----------------------------------------
Time Step:  133
Prey:  2278
Intermediate-Predator:  1495
Predator:  257
Empty:  5970
-----------------------------------------
Time Step:  134
Prey:  2377
Intermediate-Predator:  1543
Predator:  241
Empty:  5839
-----------------------------------------
Time Step:  135
Prey:  2438
Intermediate-Predator:  1590
Predator:  244
Empty:  5728
-----------------------------------------
Time Step:  136
Prey:  2480
Intermediate-Predator:  1647
Predator:  242
Empty:  5631
-----------------------------------------
Time Step:  137
Prey:  2571
Intermediate-Predator:  1676
Predator:  254
Empty:  5499
-----------------------------------------
Time Step:  138
Prey:  2714
Intermediate-Predator:  1728
Predator:  253
Empty:  5305
-----------------------------------------
Time Step:  139
Prey:  2765
Intermediate-Predator:  1763
Predator:  253
Empty:  5219
--------------------------

Time Step:  198
Prey:  2745
Intermediate-Predator:  2274
Predator:  388
Empty:  4593
-----------------------------------------
Time Step:  199
Prey:  2696
Intermediate-Predator:  2289
Predator:  347
Empty:  4668
-----------------------------------------
Time Step:  200
Prey:  2621
Intermediate-Predator:  2315
Predator:  338
Empty:  4726
-----------------------------------------
Time Step:  201
Prey:  2570
Intermediate-Predator:  2295
Predator:  336
Empty:  4799
-----------------------------------------
Time Step:  202
Prey:  2525
Intermediate-Predator:  2292
Predator:  349
Empty:  4834
-----------------------------------------
Time Step:  203
Prey:  2478
Intermediate-Predator:  2289
Predator:  347
Empty:  4886
-----------------------------------------
Time Step:  204
Prey:  2426
Intermediate-Predator:  2247
Predator:  370
Empty:  4957
-----------------------------------------
Time Step:  205
Prey:  2364
Intermediate-Predator:  2189
Predator:  410
Empty:  5037
--------------------------

Time Step:  264
Prey:  2162
Intermediate-Predator:  2049
Predator:  350
Empty:  5439
-----------------------------------------
Time Step:  265
Prey:  2188
Intermediate-Predator:  2005
Predator:  364
Empty:  5443
-----------------------------------------
Time Step:  266
Prey:  2179
Intermediate-Predator:  1953
Predator:  341
Empty:  5527
-----------------------------------------
Time Step:  267
Prey:  2166
Intermediate-Predator:  1921
Predator:  323
Empty:  5590
-----------------------------------------
Time Step:  268
Prey:  2234
Intermediate-Predator:  1837
Predator:  318
Empty:  5611
-----------------------------------------
Time Step:  269
Prey:  2306
Intermediate-Predator:  1771
Predator:  328
Empty:  5595
-----------------------------------------
Time Step:  270
Prey:  2304
Intermediate-Predator:  1802
Predator:  324
Empty:  5570
-----------------------------------------
Time Step:  271
Prey:  2302
Intermediate-Predator:  1845
Predator:  321
Empty:  5532
--------------------------

Time Step:  330
Prey:  2417
Intermediate-Predator:  1859
Predator:  444
Empty:  5280
-----------------------------------------
Time Step:  331
Prey:  2443
Intermediate-Predator:  1843
Predator:  422
Empty:  5292
-----------------------------------------
Time Step:  332
Prey:  2467
Intermediate-Predator:  1820
Predator:  414
Empty:  5299
-----------------------------------------
Time Step:  333
Prey:  2476
Intermediate-Predator:  1828
Predator:  417
Empty:  5279
-----------------------------------------
Time Step:  334
Prey:  2474
Intermediate-Predator:  1887
Predator:  427
Empty:  5212
-----------------------------------------
Time Step:  335
Prey:  2483
Intermediate-Predator:  1894
Predator:  444
Empty:  5179
-----------------------------------------
Time Step:  336
Prey:  2514
Intermediate-Predator:  1912
Predator:  431
Empty:  5143
-----------------------------------------
Time Step:  337
Prey:  2538
Intermediate-Predator:  1909
Predator:  408
Empty:  5145
--------------------------

Time Step:  396
Prey:  2266
Intermediate-Predator:  2082
Predator:  371
Empty:  5281
-----------------------------------------
Time Step:  397
Prey:  2285
Intermediate-Predator:  2027
Predator:  373
Empty:  5315
-----------------------------------------
Time Step:  398
Prey:  2289
Intermediate-Predator:  1980
Predator:  377
Empty:  5354
-----------------------------------------
Time Step:  399
Prey:  2297
Intermediate-Predator:  1933
Predator:  378
Empty:  5392
-----------------------------------------
Time Step:  400
Prey:  2346
Intermediate-Predator:  1911
Predator:  350
Empty:  5393
-----------------------------------------
Time Step:  401
Prey:  2346
Intermediate-Predator:  1902
Predator:  338
Empty:  5414
-----------------------------------------
Time Step:  402
Prey:  2369
Intermediate-Predator:  1907
Predator:  305
Empty:  5419
-----------------------------------------
Time Step:  403
Prey:  2373
Intermediate-Predator:  1902
Predator:  294
Empty:  5431
--------------------------

Time Step:  462
Prey:  2458
Intermediate-Predator:  2040
Predator:  246
Empty:  5256
-----------------------------------------
Time Step:  463
Prey:  2431
Intermediate-Predator:  2102
Predator:  239
Empty:  5228
-----------------------------------------
Time Step:  464
Prey:  2445
Intermediate-Predator:  2071
Predator:  247
Empty:  5237
-----------------------------------------
Time Step:  465
Prey:  2448
Intermediate-Predator:  2054
Predator:  242
Empty:  5256
-----------------------------------------
Time Step:  466
Prey:  2432
Intermediate-Predator:  2038
Predator:  255
Empty:  5275
-----------------------------------------
Time Step:  467
Prey:  2426
Intermediate-Predator:  2041
Predator:  268
Empty:  5265
-----------------------------------------
Time Step:  468
Prey:  2428
Intermediate-Predator:  2048
Predator:  267
Empty:  5257
-----------------------------------------
Time Step:  469
Prey:  2438
Intermediate-Predator:  2032
Predator:  263
Empty:  5267
--------------------------

Time Step:  527
Prey:  2600
Intermediate-Predator:  2013
Predator:  426
Empty:  4961
-----------------------------------------
Time Step:  528
Prey:  2590
Intermediate-Predator:  1966
Predator:  439
Empty:  5005
-----------------------------------------
Time Step:  529
Prey:  2590
Intermediate-Predator:  1985
Predator:  433
Empty:  4992
-----------------------------------------
Time Step:  530
Prey:  2521
Intermediate-Predator:  2025
Predator:  452
Empty:  5002
-----------------------------------------
Time Step:  531
Prey:  2459
Intermediate-Predator:  2084
Predator:  466
Empty:  4991
-----------------------------------------
Time Step:  532
Prey:  2493
Intermediate-Predator:  2097
Predator:  465
Empty:  4945
-----------------------------------------
Time Step:  533
Prey:  2510
Intermediate-Predator:  2097
Predator:  473
Empty:  4920
-----------------------------------------
Time Step:  534
Prey:  2485
Intermediate-Predator:  2069
Predator:  477
Empty:  4969
--------------------------

SystemExit: 0